In [57]:
!pip install llama-index -q

In [58]:
#things to run on command line
#curl -fsSL https://ollama.com/install.sh | sh
#ollama serve
#ollama pull mistral

In [59]:
!pip install llama-index-llms-ollama -q

In [60]:
!pip install pycoingecko -q

In [61]:
!pip install duckduckgo_search -q

In [90]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model = "phi3", temperature = 0.3, request_timeout = 600, system_prompt = "You are a highly responsible AI assistant specializing in cyber technology. Provide accurate, detailed, and concise responses. If confident, answer directly; if unsure, use available tools like the search_tool for real-time info, fetch_price_tool for real time cryptocurrency prices, or relevant FunctionTool for specialized queries. Prioritize accuracy, clearly indicate when using external sources, and acknowledge limitations. Maintain clarity, professionalism, and ensure explanations are easy to understand without unnecessary jargon. Avoid printing any irrelevant or incorrect information." )

In [91]:
# from llama_index.core.tools import FunctionTool
# from pycoingecko import CoinGeckoAPI
# from duckduckgo_search import DDGS
# from typing import Optional, List, Dict

# def multiply(a: int, b: int) -> int:
#     """Multiple two integers and returns the result integer"""
#     return a * b


# multiply_tool = FunctionTool.from_defaults(fn=multiply)

# def add(a: int, b: int) -> int:
#     """Add two integers and returns the result integer"""
#     return a + b


# add_tool = FunctionTool.from_defaults(fn=add)

# cg = CoinGeckoAPI()

# def fetch_price(crypto_id: str, vs_currency: str = 'usd') -> Optional[float]:
#     """
#     This function can be used to fetch the current price of a specified cryptocurrency.
    
#     :param crypto_id: The ID of the cryptocurrency. 
#                       Spaces are replaced with hyphens (e.g., 'shiba inu' -> 'shiba-inu').
#     :param vs_currency: The fiat currency to compare against (default is 'usd').
#     :return: The current price of the cryptocurrency in the specified fiat currency,
#              or None if the cryptocurrency is not found.
#     """
#     crypto_id = crypto_id.replace(" ", "-").lower()
#     price_data = cg.get_price(ids=crypto_id, vs_currencies=vs_currency)
#     return price_data.get(crypto_id, {}).get(vs_currency)

# fetch_price_tool = FunctionTool.from_defaults(fn=fetch_price)

# def duckduckgo_full_search(
#     query: str,
#     region: Optional[str] = "wt-wt",
#     max_results: Optional[int] = 3,
# ) -> List[Dict]:
#     """
#     If you don't know anything about what is being asked in the question or you are not sure of,
#     Make a query to DuckDuckGo search to receive full search results.

#     Args:
#         query (str): The query to be passed to DuckDuckGo.
#         region (Optional[str]): The region to be used for the search in [country-language] convention, ex us-en, uk-en, ru-ru, etc.
#         max_results (Optional[int]): The maximum number of results to be returned.

#     Returns:
#         List[Dict]: A list of dictionaries containing the search results.
#     """
#     params = {
#         "keywords": query,
#         "region": region,
#         "max_results": max_results,
#     }

#     with DDGS() as ddg:
#         return list(ddg.text(**params))

# search_tool = FunctionTool.from_defaults(fn=duckduckgo_full_search)

In [104]:
from llama_index.core.tools import FunctionTool
from pycoingecko import CoinGeckoAPI
from duckduckgo_search import DDGS
from typing import Optional, List, Dict
import requests

def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)


def fetch_price(crypto_id: str, vs_currency: str = 'usd') -> Optional[float]:
    """
    Fetch the current price of a specified cryptocurrency using CoinGecko API.
    
    :param crypto_id: The name of the cryptocurrency. 
                      Spaces are replaced with hyphens (e.g., 'shiba inu' -> 'shiba-inu').
    :param vs_currency: The fiat currency to compare against (default is 'usd').
    :return: The current price of the cryptocurrency in the specified fiat currency,
             or None if the cryptocurrency is not found.
    """
    crypto_id = crypto_id.replace(" ", "-").lower()
    url = "https://api.coingecko.com/api/v3/simple/price"
    headers = {"accept": "application/json"}
    params = {
        'ids': crypto_id,
        'vs_currencies': vs_currency
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        price_data = response.json()
        return price_data.get(crypto_id, {}).get(vs_currency)
    else:
        return None

fetch_price_tool = FunctionTool.from_defaults(fn=fetch_price)



def duckduckgo_full_search(
    query: str,
    region: Optional[str] = "wt-wt",
    max_results: Optional[int] = 3,
) -> List[Dict]:
    """
    If you don't know anything about what is being asked in the question or you are not sure of,
    Make a query to DuckDuckGo search to receive full search results.

    Args:
        query (str): The query to be passed to DuckDuckGo.
        region (Optional[str]): The region to be used for the search in [country-language] convention, ex us-en, uk-en, ru-ru, etc.
        max_results (Optional[int]): The maximum number of results to be returned.

    Returns:
        List[Dict]: A list of dictionaries containing the search results.
    """
    params = {
        "keywords": query,
        "region": region,
        "max_results": max_results,
    }

    with DDGS() as ddg:
        return list(ddg.text(**params))

search_tool = FunctionTool.from_defaults(fn=duckduckgo_full_search)

In [105]:
from llama_index.core.agent import ReActAgent, FunctionCallingAgentWorker

react_agent = ReActAgent.from_tools(tools=[multiply_tool, add_tool, fetch_price_tool, search_tool], llm=llm, verbose=True)

react_agent

In [106]:
while True:
    user_input = input("You: ")  
    if user_input.lower() in ['exit', 'quit', 'stop']:
        print("Goodbye!")
        break
    
    response = react_agent.chat(user_input) 
    print("Bot:", response)

Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
Thought: The user is asking for recent news about Apple Inc., which likely involves current events or developments within the company, its products, services, or market performance that have been reported in reliable sources like major newspapers and financial websites as of today's date (2024). To provide accurate information without using tools directly would require access to real-time data. However, I can simulate a response based on general knowledge up until my last update in 2023 or use the available tool 'duckduckgo_full_search'.
Action: duckduckgo_full_search
Action Input: {'query': 'latest news about Apple company'}
Observation: [{'title': 'Company News - Newsroom - Apple', 'href': 'https://www.apple.com/newsroom/topics/company-news/', 'body': 'Apple, Google partner on an ind

In [107]:
# https://github.com/run-llama/llama_index/discussions/8363#discussioncomment-7366542